# 2-task
## Описание полей

- `variable_1` - агрегированный коэффициент бонус-малус (повышающий или понижающий стоимость полиса в зависимости от аварийности в предыдущие периоды);
- `variable_2` - индикатор расторжения договора по инициативе страхователя (клиента);
- `variable_3` - индикатор расторжения договора по инициативе страховщика (страховой компании);
- `variable_4` - идентификатор года выпуска транспортного средства;
- `variable_5` - идентификатор страны - производителя транспортного средства;
- `variable_6` - мощность двигателя в лошадиных силах;
- `variable_7` - объем двигателя в куб. см;
- `variable_8` - идентификатор стороны расположения руля (левый или правый);
- `variable_9` - пробег транспортного средства, покрываемый гарантией производителя;
- `variable_10` - индикатор действия гарантии на транспортное средство;
- `variable_11` - "мультидрайв" - индикатор допуска к управлению транспортным средством более одного водителя;
- `variable_12` - возраст транспортного средства (в мес.);
- `variable_13` - возраст водителя с максимальным стажем; (если null-> target=1)
- `variable_14` - коэффициент возраст-стаж;
- `variable_15` - коэффициент краткосрочности;
- `variable_16` - коэффициент мощности;
- `variable_17` - коэффициент "мультидрайв";
- `variable_18` - территориальный коэффициент;
- `variable_19` - коэффициент "КНДР";
- `variable_20` - идентификатор канала продаж;
- `variable_21` - марка транспортного средства;
- `variable_22` - модель транспортного средства;
- `variable_23` - индикатор отечественных транспортных средств;
- `variable_24` - пол водителя с максимальным коэффициентом "возраст-стаж";
- `variable_25` - индикатор пролонгации;
- `variable_26` - индикатор совпадения собственника транспортного средства и водителя;
- `variable_27` - стаж водителя с максимальным коэффициентом "возраст-стаж";(если null-> target=1)
- `variable_28` - тип транспортного средства;
- <font color="red">`target` - равный 1, если заключение договора с клиентом является рисковым, и 0 в противном случае

<b>Метрика - ROC-AUC</b>

Одной из особенностей данной задачи является то, что почти для всех марок присутствуют разные модели, представляющие их.
https://www.kaggle.com/c/finunichamp2018-2 

_____

## Импорт библиотек и загрузка данных

In [3]:
from matplotlib import pyplot
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


PATH_TO_DATA="/Users/roman/finunichamp2018/2/data/based/"
PATH_TO_OUT="/Users/roman/finunichamp2018/2/data/"

In [4]:
train=pd.read_csv(PATH_TO_DATA+"insclass_train.csv")
test=pd.read_csv(PATH_TO_DATA+"insclass_test.csv")
sample_submit=pd.read_csv(PATH_TO_DATA+"insclass_sample.csv")
train.head(2)

,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,variable_10,...,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28,target
0,w200,0,0,14,q2,98.0,NaN,0.0,NaN,0,...,C,j2,h45,0,0.0,0,1,19.323463,t1,0
1,w160,0,0,7,q11,106.0,NaN,0.0,NaN,0,...,C,j33,h234,0,1.0,0,1,41.177900,t1,0


## EDA

In [5]:
train["target"].value_counts(normalize=True)

0    0.904502
1    0.095498
Name: target, dtype: float64

Классы не сбалансированны

In [6]:
train.shape

(151406, 29)

In [7]:
#Сколько процентов данных пропущенно
print("Какой процент данных пропущен")
for col in train.columns:
    count=train[col].count()
    if count!=train.shape[0]:
        nan=round(1-count/train.shape[0],4)
        print(col+" "+str(nan))
    

Какой процент данных пропущен
variable_6 0.0007
variable_7 0.6317
variable_8 0.1975
variable_9 0.8879
variable_12 0.0101
variable_13 0.0109
variable_14 0.0193
variable_15 0.9999
variable_16 0.0001
variable_17 0.0001
variable_18 0.0001
variable_20 0.0
variable_24 0.0146
variable_27 0.0137


### Как делать обаботку для каждго признака
- `variable_1` - <b>OHE (Загуглить что он значит и что возно выцепить из него)</b>
- `variable_2` - <b>OHE-линейн,деревянные-оставить[0/1]</b>
- `variable_3` - <b>Выбросить(не брать) тк всегда = 0 (одно значение всегда и на test и на train)</b>
- `variable_4` - <b>OHE</b> или оставить (это год выпуска машины хз как они там это кодировали мож порядок и не сохранился поэтому попробовать и то ито)
- `variable_5` - <b>OHE</b>
- `variable_6` - <b>Оставить как есть (мощность двигателя коректно сравнивать на > <) </b>
- `variable_7` - <b>Оставить как есть </b>
- `variable_8` - <b>OHE-линейн,деревянные-оставить[0/1]</b>
- `variable_9` - <b>Оставить как есть</b>
- `variable_10`- <b>OHE-линейн,деревянные-оставить[0/1]</b>
- `variable_11`- <b>OHE-линейн,деревянные-оставить[0/1]</b>
- `variable_12`- <b>Оставить и сделать метапризнак возраст в годах</b>
- `variable_13`- <b>Оставить</b>
- `variable_14`- <b>OHE-линейн,деревянные-оставить</b> или оставить(хоть он и численный но значений не много)
- `variable_15`- <b>OHE</b> или оставить(хоть он и численный но значений не много)
- `variable_16`- <b>OHE-lin оставить-tree </b>
- `variable_17`- <b>OHE</b> или оставить(хоть он и численный но значений не много)
- `variable_18`- <b>OHE</b> или оставить(хоть он и численный но значений не много)
- `variable_19`- <b>OHE</b> или оставить(хоть он и численный но значений не много)
- `variable_20`- <b>OHE</b>
- `variable_21`- OHE(много) или как есть (попробрвать взаимодействие с variable_22) <b>OHE-линейн,деревянные-оставить</b>
- `variable_22`- OHE(много) или как есть (попробрвать взаимодействие с variable_21) <b>OHE-линейн,деревянные-оставить</b>
- `variable_23`- <b>OHE-линейн,деревянные-оставить[0/1]</b>
- `variable_24`- <b>OHE-lin оставить-tree </b>
- `variable_25`- <b>OHE-линейн,деревянные-оставить[0/1]</b>
- `variable_26`- <b>OHE-линейн,деревянные-оставить[0/1]</b>
- `variable_27`- <b>OHE-lin оставить-tree </b> или оставить как есть
- `variable_28`- <b>OHE</b>


#### Обработка пропусков
- Для каждого сделать метапризнак 1-значение есть 0-зачения нет(кроме 20)
- Для деревьев пропуски заполнить значениями вне распределения
- Для линейных пропуски заполнить 
- `variable_6` -<b>заполнить lin and tree</b>
- `variable_7` -заполнить или <b>удалить тк сильно от 6 зависит</b>
- `variable_8` -<b>заполнить(tree-NAN=3 lin-NAN=Отдельный солбец)</b>
- `variable_9` -заполнить или <b>удалить так есть отриц и оооцень большые знач</b>
- `variable_12`-<b>заполнить(tree-NAN=-100 lin-NAN=среднее)</b>
- `variable_13`-<b>заполнить(tree-NAN=0 lin-NAN=Отдельный солбец)</b>
- `variable_14`-<b>заполнить(tree-NAN=0 lin-NAN=Отдельный солбец)</b>
- `variable_15`-<b>удалить(заменить все значения на есть значение-0 или нет)</b>
- `variable_16`-<b>заполнить</b>
- `variable_17`-<b>заполнить</b>
- `variable_18`-<b>заполнить</b>
- `variable_20`-<b>удалить всего одна строчка</b>
- `variable_24`-<b>заполнить(tree-NAN=3 lin-NAN=Отдельный солбец)</b>
- `variable_27`-<b>заполнить(tree-NAN=-100 lin-NAN=Отдельный солбец)</b>

## Подготовка датасетов

In [11]:
#Удаление пропусков в некоторых признаках

#тк пропусков большинство то просто сделаю флаг есть значение или нет
train["variable_15"]=train["variable_15"].isnull().astype("int")
test["variable_15"]=test["variable_15"].isnull().astype("int")

# Удалил единственный пропуск в train["variable_20"]
# train[train["variable_20"].isnull()].index return 109113
train.drop(axis=0,index=109113,inplace=True)

In [12]:
#Обьединяю train и test для обработки
target=train["target"]
train.drop("target",axis=1,inplace=True)

test.drop("id",axis=1,inplace=True)

sep=train.shape[0]

df=pd.concat([train,test])

In [ ]:
sep

_____

In [204]:
#Создание признаков и обработка пропусков

In [205]:
variable_1_both=pd.get_dummies(df["variable_1"],prefix="var_1")
variable_1_cat=df["variable_1"]

In [206]:
variable_2_lin=pd.get_dummies(df["variable_2"],prefix="var_2")
variable_2_tree=df["variable_2"]
variable_2_xgb=variable_2_tree
variable_2_cat=variable_2_tree

In [207]:
variable_4_OHE_both=pd.get_dummies(df["variable_4"],prefix="var_4")
variable_4_cat=df["variable_4"]

In [208]:
variable_5_both=pd.get_dummies(df["variable_5"],prefix="var_5")
variable_5_cat=df["variable_5"]

In [209]:
variable_6_lin=df["variable_6"].fillna(value=df["variable_6"].mean()) #не совсем правильно считать среднее и по тесту но...
variable_6_tree=df["variable_6"].fillna(value=3500) #max value 2000
variable_6_xgb=df["variable_6"]
variable_6_cat=variable_6_tree

In [210]:
# variable_7_both=df["variable_7"] Сильно корелирует с 6

In [211]:
variable_8_lin=pd.get_dummies(df["variable_8"],prefix="var_8",dummy_na=True) #Отдельный признак NAN
variable_8_tree=df["variable_8"].fillna(value=3) # 3-означает NaN
variable_8_xgb=df["variable_8"]
variable_8_cat=variable_8_tree

In [212]:
# variable_9_both=df["variable_9"] Много пропусков отризательные знач и очень большие

In [213]:
variable_10_lin=pd.get_dummies(df["variable_10"],prefix="var_10")
variable_10_tree=df["variable_10"]
variable_10_xgb=variable_10_tree
variable_10_cat=variable_10_tree

In [214]:
variable_11_lin=pd.get_dummies(df["variable_11"],prefix="var_11")
variable_11_tree=df["variable_11"]
variable_11_xgb=variable_11_tree
variable_11_cat=variable_11_tree

In [215]:
variable_12_lin_month=df['variable_12'].fillna(df['variable_12'].mean())
variable_12_lin_year=df['variable_12'].fillna(df['variable_12'].mean())/12

variable_12_tree_month=df['variable_12'].fillna(-100)
variable_12_tree_year=df['variable_12'].fillna(-100)/12

variable_12_xgb_month=df['variable_12']
variable_12_xgb_year=df['variable_12']

variable_12_cat_month=variable_12_tree_month
variable_12_cat_year=variable_12_tree_year

In [216]:
#Для lin флаг пропуска а сами пропуски=0 (вне распределения)
tmp=pd.concat([df['variable_13'],df['variable_13'].isnull().astype("int")],axis=1)
tmp.columns=['variable_13','variable_13_nan']
tmp["variable_13"].fillna(0,inplace=True)
variable_13_lin=tmp

variable_13_tree=df['variable_13'].fillna(0) # min 20
variable_13_xgb=df['variable_13']
variable_13_cat=variable_13_tree

In [217]:
variable_14_lin=pd.get_dummies(df["variable_14"],prefix="var_14",dummy_na=True)
variable_14_tree=df['variable_14'].fillna(0)
variable_14_xgb=df['variable_14']
variable_14_cat=variable_14_tree

variable_15_lin=pd.get_dummies(df["variable_15"],prefix="var_15")
variable_15_tree=df["variable_15"]
variable_15_xgb=variable_15_tree
variable_15_cat=variable_15_tree

variable_16_lin=pd.get_dummies(df["variable_16"],prefix="var_16",dummy_na=True)#Отдельное признак для nan
variable_16_tree=df["variable_16"].fillna(-100) #Для деревьев зполнил пропуски значениями вне распределения
variable_16_xgb=df["variable_16"]
variable_16_cat=variable_16_tree

variable_17_both=pd.get_dummies(df["variable_17"],prefix="var_17",dummy_na=True)
variable_17_cat=df["variable_17"]

variable_18_both=pd.get_dummies(df["variable_18"],prefix="var_18",dummy_na=True)
variable_18_cat=df["variable_18"]

variable_19_both=pd.get_dummies(df["variable_19"],prefix="var_19")
variable_19_cat=df["variable_19"]

variable_20_both=pd.get_dummies(df["variable_20"],prefix="var_20")
variable_20_cat=df["variable_20"]

In [218]:
le=LabelEncoder()

#Не очень коректно особенно для деревьев (возможно лучше выкинуть)
variable_21_lin=pd.get_dummies(df["variable_21"],prefix="var_21")
variable_21_tree=pd.Series(le.fit_transform(df["variable_21"]),name="variable_21",index=df["variable_21"].index)
variable_21_xgb=variable_21_tree
variable_21_cat=variable_21_tree

variable_22_lin=pd.get_dummies(df["variable_22"],prefix="var_22")
variable_22_tree=pd.Series(le.fit_transform(df["variable_22"]),name="variable_22",index=df["variable_22"].index)
variable_22_xgb=variable_22_tree
variable_22_cat=variable_22_tree

In [219]:
variable_23_lin=pd.get_dummies(df["variable_23"],prefix="var_23")
variable_23_tree=df["variable_23"]
variable_23_xgb=variable_23_tree
variable_23_cat=variable_23_tree

In [220]:
variable_24_lin=pd.get_dummies(df["variable_24"],prefix="var_24",dummy_na=True)
variable_24_tree=df["variable_24"].fillna(value=3)
variable_24_xgb=df["variable_24"]
variable_24_cat=variable_24_tree

In [221]:
variable_25_lin=pd.get_dummies(df["variable_25"],prefix="var_25")
variable_25_tree=df["variable_25"]
variable_25_xgb=variable_25_tree
variable_25_cat=variable_25_tree

variable_26_lin=pd.get_dummies(df["variable_26"],prefix="var_26")
variable_26_tree=df["variable_26"]
variable_26_xgb=variable_26_tree
variable_26_cat=variable_26_tree

In [222]:
variable_27_lin=pd.get_dummies(df["variable_27"],prefix="var_27",dummy_na=True)
variable_27_tree=df["variable_27"].fillna(value=-100) #min=0
variable_27_xgb=df["variable_27"]
variable_27_cat=variable_27_tree

variable_28_both=pd.get_dummies(df["variable_28"],prefix="var_28")
variable_28_cat=df["variable_28"]

In [223]:
#Формирование датасетов

#взять все variable c lin и both
cleared_linear =pd.concat([variable_1_both,
                           variable_2_lin,
                           variable_4_OHE_both,
                           variable_5_both,
                           variable_6_lin,
                           variable_8_lin,
                           variable_10_lin,
                           variable_11_lin,
                           variable_12_lin_month,
                           variable_12_lin_year,
                           variable_13_lin,
                           variable_14_lin,
                           variable_15_lin,
                           variable_16_lin,
                           variable_17_both,
                           variable_18_both,
                           variable_19_both,
                           variable_20_both,
                           variable_21_lin,
                           variable_22_lin,
                           variable_23_lin,
                           variable_24_lin,
                           variable_25_lin,
                           variable_26_lin,
                           variable_27_lin,
                           variable_28_both],axis=1)

#взять все variable c tree и both
cleared_tree =pd.concat([variable_1_both,
                         variable_2_tree,
                         variable_4_OHE_both,
                         variable_5_both,
                         variable_6_tree,
                         variable_8_tree,
                         variable_10_tree,
                         variable_11_tree,
                         variable_12_tree_month,
                         variable_12_tree_year,
                         variable_13_tree,
                         variable_14_tree,
                         variable_15_tree,
                         variable_16_tree,
                         variable_17_both,
                         variable_18_both,
                         variable_19_both,
                         variable_20_both,
                         variable_21_tree,
                         variable_22_tree,
                         variable_23_tree,
                         variable_24_tree,
                         variable_25_tree,
                         variable_26_tree,
                         variable_27_tree,
                         variable_28_both],axis=1)

#взять все variable c xgb и both
cleared_xgb=pd.concat([variable_1_both,
                       variable_2_xgb,
                       variable_4_OHE_both,
                       variable_5_both,
                       variable_6_xgb,
                       variable_8_xgb,
                       variable_10_xgb,
                       variable_11_xgb,
                       variable_12_xgb_month,
                       variable_12_xgb_year,
                       variable_13_xgb,
                       variable_14_xgb,
                       variable_15_xgb,
                       variable_16_xgb,
                       variable_17_both,
                       variable_18_both,
                       variable_19_both,
                       variable_20_both,
                       variable_21_xgb,
                       variable_22_xgb,
                       variable_23_xgb,
                       variable_24_xgb,
                       variable_25_xgb,
                       variable_26_xgb,
                       variable_27_xgb,
                       variable_28_both],axis=1)

#взять все variable c cat
cleared_cat=pd.concat([variable_1_cat,
                       variable_2_cat,
                       variable_4_cat,
                       variable_5_cat,
                       variable_6_cat,
                       variable_8_cat,
                       variable_10_cat,
                       variable_11_cat,
                       variable_12_cat_month,
                       variable_12_cat_year,
                       variable_13_cat,
                       variable_14_cat,
                       variable_15_cat,
                       variable_16_cat,
                       variable_17_cat,
                       variable_18_cat,
                       variable_19_cat,
                       variable_20_cat,
                       variable_21_cat,
                       variable_22_cat,
                       variable_23_cat,
                       variable_24_cat,
                       variable_25_cat,
                       variable_26_cat,
                       variable_27_cat,
                       variable_28_cat],axis=1)

## Сохранение датафреймов для дальнейшей работы

In [224]:
cleared_linear.to_csv(PATH_TO_OUT+"cleared_linear.csv",index=False)

In [225]:
cleared_tree.to_csv(PATH_TO_OUT+"cleared_tree.csv",index=False)

In [226]:
cleared_xgb.to_csv(PATH_TO_OUT+"cleared_xgb.csv",index=False)

In [227]:
cleared_cat.to_csv(PATH_TO_OUT+"cleared_cat.csv",index=False)

In [228]:
target.to_csv(PATH_TO_OUT+"target.csv",header=True,index=False)